# Extracción Optimizada de Atributos con Gemini

Este notebook extrae atributos de productos a partir de imágenes usando Google Gemini API.

## Características:
- Extracción desde CSV
- Guardado automático en CSV
- Reintentos automáticos con backoff exponencial
- Reanudación automática desde último punto
- Barra de progreso visual
- Logging detallado

In [18]:
import os
import time
import logging
from pathlib import Path
from typing import Optional, Dict, Any
from datetime import datetime

import pandas as pd
from google import genai
from google.genai import types
from tqdm.auto import tqdm
from dotenv import load_dotenv

## 1. Configuración

In [19]:
# Configuración del logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('extraccion_atributos.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Cargar variables de entorno
load_dotenv()

# Configuración de rutas
class Config:
    """Configuración centralizada del proyecto"""
    
    # Rutas
    PROMPT_FILE = Path('prompt_api.txt')
    IMAGE_DIRECTORY = Path('images')  # Carpeta con las imágenes
    INPUT_CSV = Path('productos_coppel.csv')  # CSV de entrada con columnas: id, image, metadatos
    OUTPUT_CSV = Path('productos_con_atributos.csv')  # CSV de salida
    
    # API Configuration
    GEMINI_MODEL = 'gemini-2.5-flash'  # Modelo más reciente y rápido
    MAX_RETRIES = 5
    BASE_DELAY = 5  # segundos
    RATE_LIMIT_DELAY = 1.5  # segundos entre llamadas
    
    # Columnas esperadas en el CSV
    ID_COLUMN = 'id'
    IMAGE_COLUMN = 'path_image'
    ATTRIBUTES_COLUMN = 'gemini_attributes'

config = Config()
logger.info(f"Configuración cargada: {config.__dict__}")

2025-10-15 16:09:59,749 - INFO - Configuración cargada: {}


In [25]:
# Inicializar cliente de Gemini
try:
    client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))
    logger.info("Cliente de Gemini inicializado correctamente")
    
except Exception as e:
    logger.error(f"Error al inicializar cliente de Gemini: {e}")
    raise

2025-10-15 16:10:40,124 - INFO - Cliente de Gemini inicializado correctamente


## 2. Funciones Auxiliares

In [21]:
def load_prompt(file_path: Path) -> str:
    """Carga el texto del prompt desde un archivo."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            prompt = f.read().strip()
        logger.info(f"Prompt cargado desde {file_path} ({len(prompt)} caracteres)")
        return prompt
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='latin-1') as f:
            prompt = f.read().strip()
        logger.warning(f"Prompt cargado con encoding latin-1")
        return prompt
    except Exception as e:
        logger.error(f"Error al cargar prompt: {e}")
        raise


def get_mime_type(image_path: Path) -> str:
    """Determina el tipo MIME de la imagen."""
    extension = image_path.suffix.lower()
    mime_types = {
        '.jpg': 'image/jpeg',
        '.jpeg': 'image/jpeg',
        '.png': 'image/png',
        '.gif': 'image/gif',
        '.webp': 'image/webp'
    }
    return mime_types.get(extension, 'image/jpeg')


def process_image_with_gemini(
    image_path: Path,
    prompt_text: str,
    max_retries: int = config.MAX_RETRIES,
    base_delay: int = config.BASE_DELAY
) -> str:
    """
    Procesa una imagen con Gemini API y retorna los atributos extraídos.
    
    Args:
        image_path: Ruta a la imagen
        prompt_text: Texto del prompt
        max_retries: Número máximo de reintentos
        base_delay: Delay base para backoff exponencial
        
    Returns:
        String con los atributos extraídos o mensaje de error
    """
    if not image_path.exists():
        return f"ERROR_IMAGEN: Archivo no encontrado en {image_path}"
    
    # Leer imagen
    try:
        with open(image_path, 'rb') as f:
            image_bytes = f.read()
    except Exception as e:
        return f"ERROR_LECTURA: {str(e)}"
    
    mime_type = get_mime_type(image_path)
    
    # Preparar contenido para Gemini
    contents = [
        types.Part.from_bytes(data=image_bytes, mime_type=mime_type),
        types.Part.from_text(text=prompt_text)
    ]
    
    # Reintentos con backoff exponencial
    for attempt in range(max_retries):
        try:
            response = client.models.generate_content(
                model=config.GEMINI_MODEL,
                contents=contents
            )
            return response.text.strip().replace('\n', ' ')
            
        except Exception as e:
            error_message = str(e)
            logger.warning(f"Intento {attempt + 1}/{max_retries} falló: {error_message}")
            
            if attempt < max_retries - 1:
                sleep_time = base_delay * (2 ** attempt)
                logger.info(f"Esperando {sleep_time}s antes de reintentar...")
                time.sleep(sleep_time)
            else:
                return f"ERROR_API_FATAL: {error_message}"
    
    return "ERROR_INESPERADO: Bucle de reintento fallido"

## 3. Procesamiento de Datos

In [22]:
def load_or_create_dataframe(input_csv: Path) -> pd.DataFrame:
    """Carga el CSV de entrada o crea uno vacío si no existe."""
    if input_csv.exists():
        df = pd.read_csv(input_csv)
        logger.info(f"CSV cargado: {len(df)} registros")
        
        # Asegurar que exista la columna de atributos
        if config.ATTRIBUTES_COLUMN not in df.columns:
            df[config.ATTRIBUTES_COLUMN] = ''
            logger.info(f"Columna '{config.ATTRIBUTES_COLUMN}' creada")
        
        return df
    else:
        logger.warning(f"Archivo {input_csv} no encontrado. Creando DataFrame vacío.")
        return pd.DataFrame(columns=[config.ID_COLUMN, config.IMAGE_COLUMN, config.ATTRIBUTES_COLUMN])


def save_checkpoint(df: pd.DataFrame, output_csv: Path) -> None:
    """Guarda el DataFrame en CSV."""
    try:
        df.to_csv(output_csv, index=False, encoding='utf-8')
        logger.info(f"Checkpoint guardado en {output_csv}")
    except Exception as e:
        logger.error(f"Error al guardar checkpoint: {e}")


def should_process_row(row: pd.Series) -> bool:
    """Determina si una fila debe ser procesada."""
    attributes_value = str(row.get(config.ATTRIBUTES_COLUMN, '')).strip()
    
    # Procesar si está vacío o es un error fatal que queremos reintentar
    if not attributes_value:
        return True
    
    # No reprocesar si ya tiene atributos válidos
    if not attributes_value.startswith('ERROR'):
        return False
    
    # Reprocesar solo errores fatales
    if 'ERROR_API_FATAL' in attributes_value:
        return True
    
    return False

## 4. Proceso Principal

In [23]:
def run_extraction(
    input_csv: Path = config.INPUT_CSV,
    output_csv: Path = config.OUTPUT_CSV,
    prompt_file: Path = config.PROMPT_FILE,
    image_dir: Path = config.IMAGE_DIRECTORY,
    save_every: int = 1  # Guardar después de cada procesamiento
) -> pd.DataFrame:
    """
    Ejecuta el proceso de extracción de atributos.
    
    Args:
        input_csv: Ruta al CSV de entrada
        output_csv: Ruta al CSV de salida
        prompt_file: Ruta al archivo de prompt
        image_dir: Directorio con las imágenes
        save_every: Guardar cada N procesados (default: 1)
        
    Returns:
        DataFrame con los atributos extraídos
    """
    logger.info("=" * 60)
    logger.info("INICIANDO EXTRACCIÓN DE ATRIBUTOS")
    logger.info("=" * 60)
    
    # Cargar prompt
    prompt = load_prompt(prompt_file)
    
    # Cargar o crear DataFrame
    df = load_or_create_dataframe(input_csv)
    
    if len(df) == 0:
        logger.warning("No hay datos para procesar")
        return df
    
    # Filtrar filas a procesar
    rows_to_process = df.apply(should_process_row, axis=1)
    total_to_process = rows_to_process.sum()
    
    logger.info(f"Total de productos: {len(df)}")
    logger.info(f"A procesar: {total_to_process}")
    logger.info(f"Ya procesados: {len(df) - total_to_process}")
    
    if total_to_process == 0:
        logger.info("Todos los productos ya están procesados")
        return df
    
    # Procesar cada fila
    processed_count = 0
    
    with tqdm(total=total_to_process, desc="Extrayendo atributos") as pbar:
        for idx, row in df.iterrows():
            if not rows_to_process[idx]:
                continue
            
            product_id = row.get(config.ID_COLUMN, idx)
            image_filename = row.get(config.IMAGE_COLUMN, '')
            
            if not image_filename:
                logger.warning(f"Fila {idx}: Sin nombre de imagen")
                df.at[idx, config.ATTRIBUTES_COLUMN] = "ERROR_SIN_IMAGEN"
                continue
            
            image_path = image_dir / image_filename
            
            logger.info(f"Procesando {product_id}: {image_filename}")
            
            # Procesar con Gemini
            attributes = process_image_with_gemini(image_path, prompt)
            
            # Guardar resultado
            df.at[idx, config.ATTRIBUTES_COLUMN] = attributes
            
            # Log resultado
            if attributes.startswith("ERROR"):
                logger.error(f"{product_id}: {attributes[:100]}")
            else:
                logger.info(f"{product_id}: {attributes[:80]}...")
            
            processed_count += 1
            pbar.update(1)
            
            # Guardar checkpoint
            if processed_count % save_every == 0:
                save_checkpoint(df, output_csv)
            
            # Rate limiting
            time.sleep(config.RATE_LIMIT_DELAY)
    
    # Guardar resultado final
    save_checkpoint(df, output_csv)
    
    # Estadísticas finales
    successful = len(df[~df[config.ATTRIBUTES_COLUMN].str.startswith('ERROR', na=False)])
    errors = len(df[df[config.ATTRIBUTES_COLUMN].str.startswith('ERROR', na=False)])
    
    logger.info("=" * 60)
    logger.info("PROCESO COMPLETADO")
    logger.info(f"Exitosos: {successful}")
    logger.info(f"Errores: {errors}")
    logger.info(f"Guardado en: {output_csv}")
    logger.info("=" * 60)
    
    return df

## 5. Ejecución

In [27]:
# Ejecutar el proceso de extracción
df_result = run_extraction()

# Mostrar primeros resultados
print("\n" + "="*60)
print("PRIMEROS 5 RESULTADOS:")
print("="*60)
df_result.head(10)

2025-10-15 16:11:39,689 - INFO - ============================================================
2025-10-15 16:11:39,692 - INFO - INICIANDO EXTRACCIÓN DE ATRIBUTOS
2025-10-15 16:11:39,692 - INFO - ============================================================
2025-10-15 16:11:39,694 - INFO - Prompt cargado desde prompt_api.txt (6895 caracteres)
2025-10-15 16:11:39,698 - INFO - CSV cargado: 24 registros
2025-10-15 16:11:39,699 - INFO - Columna 'gemini_attributes' creada
2025-10-15 16:11:39,701 - INFO - Total de productos: 24
2025-10-15 16:11:39,702 - INFO - A procesar: 24
2025-10-15 16:11:39,703 - INFO - Ya procesados: 0


Extrayendo atributos:   0%|          | 0/24 [00:00<?, ?it/s]

2025-10-15 16:11:39,712 - INFO - Procesando nan: pr-5249912-1.jpg
2025-10-15 16:11:39,714 - INFO - AFC is enabled with max remote calls: 10.
2025-10-15 16:11:59,655 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-10-15 16:11:59,658 - INFO - nan: Tipo: Pañalero, Detalles: Estampados variados y texto, Bolsillos: nan, Composici...
2025-10-15 16:11:59,672 - INFO - Checkpoint guardado en productos_con_atributos.csv
2025-10-15 16:12:01,184 - INFO - Procesando nan: pr-5226632-1.jpg
2025-10-15 16:12:01,188 - INFO - AFC is enabled with max remote calls: 10.
2025-10-15 16:12:19,825 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-10-15 16:12:19,829 - INFO - nan: Tipo: nan, Detalles: cintura elástica, Con capucha, Forro de borreguito, Detalle...
2025-10-15 16:12:19,839 - INFO - Checkpoint guardado en productos_con_at


PRIMEROS 5 RESULTADOS:


,id,name,image,price,description,brand,category,path_image,gemini_attributes
0,NaN,Pañalero para Bebé Niña Baby Colors 5 Piezas,https://cdn5.coppel.com/pr/5249912-1.jpg?iresi...,Precio de contado,NaN,NaN,Bebé,pr-5249912-1.jpg,"Tipo: Pañalero, Detalles: Estampados variados ..."
1,NaN,Chamarra Baby Colors para Bebé Niña,https://cdn5.coppel.com/pr/5226632-1.jpg?iresi...,Precio de contado,NaN,NaN,Bebé,pr-5226632-1.jpg,"Tipo: nan, Detalles: cintura elástica, Con cap..."
2,NaN,Pañaleros y Pantalón marca Baby Colors para Be...,https://cdn5.coppel.com/pr/5028582-1.jpg?iresi...,Precio de contado,NaN,NaN,Bebé,pr-5028582-1.jpg,"Tipo: nan, Detalles: cintura elástica, aplique..."
3,NaN,Baberos Impermebale Abeja,https://cdn5.coppel.com/mkp/69511265-1.jpg?ire...,Precio de contado,NaN,NaN,Bebé,mkp-69511265-1.jpg,"Tipo: nan, Detalles: Bolsillo colector de alim..."
4,NaN,Pantalón para Bebé Niño Baby Colors 3 Piezas,https://cdn5.coppel.com/pr/5198822-1.jpg?iresi...,Precio de contado,NaN,NaN,Bebé,pr-5198822-1.jpg,"Tipo: nan, Detalles: cintura elástica, puños e..."
5,NaN,Playera Baby Colors para Bebé Niño 3 Piezas,https://cdn5.coppel.com/pr/5186532-1.jpg?iresi...,Precio de contado,NaN,NaN,Bebé,pr-5186532-1.jpg,"Tipo: nan, Detalles: cintura elástica, estampa..."
6,NaN,Mameluco Bam-Bu para Bebé Niña,https://cdn5.coppel.com/pr/5237962-1.jpg?iresi...,Precio de contado,NaN,NaN,Bebé,pr-5237962-1.jpg,"Tipo: nan, Detalles: cuello y puños con volant..."
7,NaN,Kids Sofi Zapatitos Para Bebé Niña Negro Charol,https://cdn5.coppel.com/mkp/62755381-1.jpg?ire...,Precio de contado,NaN,NaN,Bebé,mkp-62755381-1.jpg,"Tipo: nan, Detalles: Lazo decorativo, cierre d..."
8,NaN,Pañalero y Pantalón Baby Colors para Bebé Niño...,https://cdn5.coppel.com/pr/5238152-1.jpg?iresi...,Precio de contado,NaN,NaN,Bebé,pr-5238152-1.jpg,"Tipo: nan, Detalles: cintura elástica, Bolsill..."
9,NaN,Conjunto para Bebé Niño Paw Patrol 3 Piezas,https://cdn5.coppel.com/pr/5248512-1.jpg?iresi...,Precio de contado,NaN,NaN,Bebé,pr-5248512-1.jpg,"Tipo: nan, Detalles: cintura elástica, Bolsill..."


## 6. Análisis de Resultados

In [ ]:
# Estadísticas de procesamiento
print("\n📊 ESTADÍSTICAS DE PROCESAMIENTO")
print("=" * 60)

total = len(df_result)
with_attributes = len(df_result[df_result[config.ATTRIBUTES_COLUMN].str.len() > 0])
errors = len(df_result[df_result[config.ATTRIBUTES_COLUMN].str.startswith('ERROR', na=False)])
successful = with_attributes - errors
pending = total - with_attributes

print(f"Total de productos: {total}")
print(f"✅ Procesados exitosamente: {successful} ({successful/total*100:.1f}%)")
print(f"❌ Con errores: {errors} ({errors/total*100:.1f}%)")
print(f"⏳ Pendientes: {pending} ({pending/total*100:.1f}%)")

# Tipos de errores
if errors > 0:
    print("\n🔍 TIPOS DE ERRORES:")
    error_df = df_result[df_result[config.ATTRIBUTES_COLUMN].str.startswith('ERROR', na=False)]
    error_types = error_df[config.ATTRIBUTES_COLUMN].str.split(':', expand=True)[0].value_counts()
    for error_type, count in error_types.items():
        print(f"  {error_type}: {count}")

## 7. Exportar Resultados Limpios

In [ ]:
# Exportar solo productos procesados exitosamente
df_clean = df_result[~df_result[config.ATTRIBUTES_COLUMN].str.startswith('ERROR', na=False)]
df_clean = df_clean[df_clean[config.ATTRIBUTES_COLUMN].str.len() > 0]

output_clean = Path('productos_limpios.csv')
df_clean.to_csv(output_clean, index=False, encoding='utf-8')

print(f"\n✅ Resultados limpios exportados a: {output_clean}")
print(f"Total de registros limpios: {len(df_clean)}")

## 8. Utilidades Adicionales

In [ ]:
# Función para reprocesar solo errores
def reprocess_errors(df: pd.DataFrame) -> pd.DataFrame:
    """Reprocesa solo los registros con errores."""
    error_mask = df[config.ATTRIBUTES_COLUMN].str.startswith('ERROR', na=False)
    df.loc[error_mask, config.ATTRIBUTES_COLUMN] = ''  # Limpiar errores
    return run_extraction()

# Descomentar para reprocesar errores
# df_result = reprocess_errors(df_result)

In [ ]:
# Función para verificar imágenes faltantes
def check_missing_images(df: pd.DataFrame, image_dir: Path) -> pd.DataFrame:
    """Verifica qué imágenes no se encuentran en el directorio."""
    missing = []
    for idx, row in df.iterrows():
        image_filename = row.get(config.IMAGE_COLUMN, '')
        if image_filename:
            image_path = image_dir / image_filename
            if not image_path.exists():
                missing.append({
                    'id': row.get(config.ID_COLUMN, idx),
                    'image': image_filename
                })
    
    missing_df = pd.DataFrame(missing)
    print(f"\n🔍 Imágenes faltantes: {len(missing)}")
    return missing_df

# Descomentar para verificar imágenes faltantes
# missing_images = check_missing_images(df_result, config.IMAGE_DIRECTORY)
# missing_images